# 使用@property

为了限制score的范围，可以通过一个set_score()方法来设置成绩，再通过一个get_score()来获取成绩，这样，在set_score()方法里，就可以检查参数：

In [10]:
class Student(object):
    
    def get_score(self):
        return self._score
    
    def set_score(self,value):
        if not isinstance(value,int):
            raise ValueError('Score must be an interger!')
        if value < 0 or value > 100:
            raise ValueError('Score must be between 0~100')
            
        self._score = value
        

s = Student()
s.set_score(90)
s.get_score()
#s.set_score(999)

90

如果要让内部属性不被外部访问，可以把属性的名称前加上两个下划线\__，在Python中，实例的变量名如果以__开头，就变成了一个私有变量（private），只有内部可以访问，外部不能访问

有些时候，你会看到以一个下划线开头的实例变量名，比如_name，这样的实例变量外部是可以访问的，但是，按照约定俗成的规定，当你看到这样的变量时，意思就是，“虽然我可以被访问，但是，请把我视为私有变量，不要随意访问”。

但是，上面的调用方法又略显复杂，没有直接用属性这么直接简单。有没有既能检查参数，又可以用类似属性这样简单的方式来访问类的变量呢？还记得装饰器（decorator）可以给函数动态加上功能吗？对于类的方法，装饰器一样起作用。Python内置的@property装饰器就是负责把一个方法变成属性调用的：

In [18]:
class Student(object):
    
    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self,value):
        if not isinstance(value,int):
            raise ValueError('Score must be an interger!')
        if value < 0 or value > 100:
            raise ValueError('Score must be between 0~100')
            
        self._score = value
        

s = Student()
s.score = 60 # 实际事转化s.set_score(60)
s.score # 实际事转化s.get_score()

# s.score = 999

60

@property的实现比较复杂，我们先考察如何使用。把一个getter方法变成属性，只需要加上@property就可以了，此时，@property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值，于是，我们就拥有一个可控的属性操作。 

注意到这个神奇的@property，我们在对实例属性操作的时候，就知道该属性很可能不是直接暴露的，而是通过getter和setter方法来实现的。

还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性：

In [21]:
class Student(object):
    
    @property
    def birth(self):
        return self._birth
    
    @birth.setter
    def birth(self,value):
        self._birth = value
    
    @property
    def age(self):
        return 2018-self._birth
    
s = Student()
s.birth = 28
s.age

1990

上面的birth是可读写属性，而age就是一个只读属性，因为age可以根据birth和当前时间计算出来。